# Layout

+ Intro algebra
    + news
    + physics
+ doc2vec
    + press releases
+ score function
    + gensim
+ dimensions example, e.g. he-she

In [1]:
#All these packages need to be installed from pip
import gensim#For word2vec, etc
import requests #For downloading our datasets
import nltk #For stop words and stemmers
import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning
%matplotlib inline

import os #For looking through files
import os.path #For managing file paths


# Intro

intro stuff ...

# Getting our corpuses

Instead of downloading our corpora, we have download them ahead of time, a subset of the [senate press releases](https://github.com/lintool/GrimmerSenatePressReleases) are in `data/grimmerPressReleases`. So we will load them into a DataFrame, to do this first we need to define a function to convert directories of text files into DataFrames.

In [2]:
def loadDir(targetDir, category):
    allFileNames = os.listdir(targetDir)
    #We need to make them into useable paths and filter out hidden files
    filePaths = [os.path.join(targetDir, fname) for fname in allFileNames if fname[0] != '.']

    #The dict that will become the DataFrame
    senDict = {
        'category' : [category] * len(filePaths),
        'filePath' : [],
        'text' : [],
    }

    for fPath in filePaths:
        with open(fPath) as f:
            senDict['text'].append(f.read())
            senDict['filePath'].append(fPath)

    return pandas.DataFrame(senDict)

Now we can use the function in all the directories in `data/grimmerPressReleases`

In [3]:
dataDir = 'data/grimmerPressReleases'

senReleasesDF = pandas.DataFrame()

for senatorName in [d for d in os.listdir(dataDir) if d[0] != '.']:
    senPath = os.path.join(dataDir, senatorName)
    senReleasesDF = senReleasesDF.append(loadDir(senPath, senatorName), ignore_index = True)

senReleasesDF[:100:10]

,category,filePath,text
0,Kennedy,data/grimmerPressReleases/Kennedy/01Apr2005Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE...
10,Kennedy,data/grimmerPressReleases/Kennedy/01Dec2005Ken...,FOR IMMEDIATE RELEASE Washington ...
20,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2006Ken...,FOR IMMEDIATE RELEASE Fact sheet...
30,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2007Ken...,FOR IMMEDIATE RELEASE Washington ...
40,Kennedy,data/grimmerPressReleases/Kennedy/01Jun2007Ken...,FOR IMMEDIATE RELEASE BOSTON MA Se...
50,Kennedy,data/grimmerPressReleases/Kennedy/01Mar2007Ken...,FOR IMMEDIATE RELEASE Washington ...
60,Kennedy,data/grimmerPressReleases/Kennedy/01May2007Ken...,FOR IMMEDIATE RELEASE The President ...
70,Kennedy,data/grimmerPressReleases/Kennedy/01Nov2007Ken...,FOR IMMEDIATE RELEASE Washington DC...
80,Kennedy,data/grimmerPressReleases/Kennedy/02Aug2006Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE ...
90,Kennedy,data/grimmerPressReleases/Kennedy/02Feb2005Ken...,FOR IMMEDIATE RELEASE The Preside...


# Stemming is taking a really long time do to the size of the dataset, so it's been disabled for now

We also want to remove stop words and stem, but tokenizing requires two steps. Word2Vec wants to know the sentence structure as well as simply the words, so the tokenizing is slightly different this time.

In [4]:
#Define the same function as last week
def normlizeTokens(tokenLst, stopwordLst = None, stemmer = None, lemmer = None):
    #We can use a generator here as we just need to iterate over it

    #Lowering the case and removing non-words
    workingIter = (w.lower() for w in tokenLst if w.isalpha())

    #Now we can use the semmer, if provided
    if stemmer is not None:
        workingIter = (stemmer.stem(w) for w in workingIter)

    #And the lemmer
    if lemmer is not None:
        workingIter = (lemmer.lemmatize(w) for w in workingIter)

    #And remove the stopwords
    if stopwordLst is not None:
        workingIter = (w for w in workingIter if w not in stopwordLst)
    #We will return a list with the stopwords removed
    return list(workingIter)

#initialize our stemmer and our stop words
stop_words_nltk = nltk.corpus.stopwords.words('english')
snowball = nltk.stem.snowball.SnowballStemmer('english')

#Apply our functions, notice each row is a list of lists now
senReleasesDF['tokenized_sents'] = senReleasesDF['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
senReleasesDF['normalized_sents'] = senReleasesDF['tokenized_sents'].apply(lambda x: [normlizeTokens(s, stopwordLst = stop_words_nltk, stemmer = None) for s in x])

senReleasesDF[:100:10]

,category,filePath,text,tokenized_sents,normalized_sents
0,Kennedy,data/grimmerPressReleases/Kennedy/01Apr2005Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE...,"[[FOR, IMMEDIATE, RELEASE, FOR, IMMEDIATE, REL...","[[immediate, release, immediate, release, cont..."
10,Kennedy,data/grimmerPressReleases/Kennedy/01Dec2005Ken...,FOR IMMEDIATE RELEASE Washington ...,"[[FOR, IMMEDIATE, RELEASE, Washington, D, C, T...","[[immediate, release, washington, c, today, se..."
20,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2006Ken...,FOR IMMEDIATE RELEASE Fact sheet...,"[[FOR, IMMEDIATE, RELEASE, Fact, sheets, on, B...","[[immediate, release, fact, sheets, bush, plan..."
30,Kennedy,data/grimmerPressReleases/Kennedy/01Feb2007Ken...,FOR IMMEDIATE RELEASE Washington ...,"[[FOR, IMMEDIATE, RELEASE, Washington, D, C, T...","[[immediate, release, washington, c, today, u,..."
40,Kennedy,data/grimmerPressReleases/Kennedy/01Jun2007Ken...,FOR IMMEDIATE RELEASE BOSTON MA Se...,"[[FOR, IMMEDIATE, RELEASE, BOSTON, MA, Senator...","[[immediate, release, boston, senator, edward,..."
50,Kennedy,data/grimmerPressReleases/Kennedy/01Mar2007Ken...,FOR IMMEDIATE RELEASE Washington ...,"[[FOR, IMMEDIATE, RELEASE, Washington, DC, Tod...","[[immediate, release, washington, dc, today, s..."
60,Kennedy,data/grimmerPressReleases/Kennedy/01May2007Ken...,FOR IMMEDIATE RELEASE The President ...,"[[FOR, IMMEDIATE, RELEASE, The, President, is,...","[[immediate, release, president, wrong, veto, ..."
70,Kennedy,data/grimmerPressReleases/Kennedy/01Nov2007Ken...,FOR IMMEDIATE RELEASE Washington DC...,"[[FOR, IMMEDIATE, RELEASE, Washington, DC, Sen...","[[immediate, release, washington, dc, senators..."
80,Kennedy,data/grimmerPressReleases/Kennedy/02Aug2006Ken...,FOR IMMEDIATE RELEASE FOR IMMEDIATE ...,"[[FOR, IMMEDIATE, RELEASE, FOR, IMMEDIATE, REL...","[[immediate, release, immediate, release, impo..."
90,Kennedy,data/grimmerPressReleases/Kennedy/02Feb2005Ken...,FOR IMMEDIATE RELEASE The Preside...,"[[FOR, IMMEDIATE, RELEASE, The, President, gav...","[[immediate, release, president, gave, effecti..."


#Word2Vec

We will be using the gensim implementation of [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec).

To load our data our data we give all the sentences to the trainer

In [5]:
senReleasesW2V = gensim.models.word2vec.Word2Vec(senReleasesDF['tokenized_sents'].sum())

Now we can look at a few things

In [6]:
senReleasesW2V.most_similar('president')

[('administration', 0.7568289041519165),
 ('President', 0.7529541850090027),
 ('Administration', 0.6696342825889587),
 ('nominee', 0.6440970301628113),
 ('himself', 0.6405242681503296),
 ('memo', 0.6264574527740479),
 ('he', 0.6256895065307617),
 ('Cheney', 0.6094998717308044),
 ('Presidents', 0.608191192150116),
 ('Pentagon', 0.599648118019104)]

Or find relationships

In [7]:
#Man is to senator as woman is to ...
senReleasesW2V.most_similar(positive = ['woman', 'senator'], negative = ['man'], topn = 5)
#Chinatown is a noble profession

[('Chinatown', 0.665317177772522),
 ('Iron', 0.6631838083267212),
 ('charity', 0.6588665246963501),
 ('Pittsfield', 0.6566770076751709),
 ('Moorhead', 0.6544506549835205)]

Get the vector

In [8]:
senReleasesW2V['president']

array([  7.30614588e-02,  -3.45240355e-01,   1.50905538e+00,
         6.34938359e-01,   2.52880603e-01,   5.29100418e-01,
         1.81260169e+00,  -2.13264036e+00,   8.48588765e-01,
         1.53643072e+00,  -9.25825119e-01,  -4.96639758e-01,
         1.75919652e+00,  -1.58743358e+00,   1.33599949e+00,
        -3.45647216e-01,   1.92963406e-01,  -6.62870556e-02,
        -1.02382481e+00,  -1.01510417e+00,  -6.69602990e-01,
         6.28593981e-01,  -6.25705183e-01,  -1.03368707e-01,
        -3.48943770e-01,   5.96535742e-01,   2.05678850e-01,
        -6.94201767e-01,  -5.83823800e-01,  -3.71038020e-01,
        -7.93978572e-01,   7.31354952e-01,   3.13704401e-01,
        -3.42074871e-01,   5.03109992e-01,  -8.74512315e-01,
        -1.61182189e+00,   5.13637960e-01,  -5.60376532e-02,
        -9.43049490e-01,   1.02671467e-01,  -1.82064816e-01,
         5.87572813e-01,   1.20939672e+00,  -1.27947581e+00,
         1.26226830e+00,   2.78864413e-01,   4.58728164e-01,
        -1.16603374e+00,

Get all the vectors

In [9]:
senReleasesW2V.syn0

array([[ 0.85325009, -0.37248051, -0.45195943, ..., -1.50720394,
        -0.36614648, -0.7876817 ],
       [ 2.36519051, -1.41433895,  1.03849816, ..., -0.97026128,
         0.23737457, -2.49554253],
       [ 0.90348792,  0.21175879, -0.4225637 , ...,  0.48453137,
         0.25113538, -0.73218578],
       ..., 
       [-0.08018535, -0.0157751 ,  0.03091644, ...,  0.04544521,
         0.00317775,  0.08965429],
       [-0.09362413,  0.06673632, -0.04862032, ...,  0.12646115,
         0.04953843, -0.01611871],
       [ 0.02434441, -0.07023816, -0.03426784, ..., -0.01972212,
         0.02808639,  0.0497155 ]], dtype=float32)

Find what doesn't fit

In [10]:
senReleasesW2V.doesnt_match(['she', 'he', 'her', 'him', 'washington'])

'her'

Or save for use later

In [11]:
senReleasesW2V.save("data/senpressreleasesWORD2Vec")

# APS abstracts

In [12]:
apsDF = pandas.read_csv('data/APSabstracts1950s.csv', index_col = 0)
apsDF['tokenized_sents'] = apsDF['abstract'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
apsDF['normalized_sents'] = apsDF['tokenized_sents'].apply(lambda x: [normlizeTokens(s, stopwordLst = stop_words_nltk, stemmer = None) for s in x])

apsW2V = gensim.models.word2vec.Word2Vec(apsDF['tokenized_sents'].sum())